In [57]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, Bidirectional, LeakyReLU
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard, CSVLogger

import os

# === Step 1: Load and preprocess ===
df = pd.read_csv("acquiredDataset.csv")

# Compute band features
df['alpha'] = (df['lowAlpha'] + df['highAlpha']) / 2
df['beta'] = (df['lowBeta'] + df['highBeta']) / 2
df = df[['alpha', 'beta', 'theta', 'delta', 'classification']]

# === Step 2: Log-transform & MinMax scale ===
features = df.drop(columns=['classification'])
labels = df['classification']

# Apply np.log1p
features_log = np.log1p(features)

# MinMax scaling
scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features_log)

# Combine with label
df_processed = pd.DataFrame(features_scaled, columns=features.columns)
df_processed['classification'] = labels.values


In [58]:

# === Step 3: Generate sequences ===
def generate_sequences(dataframe, seq_length):
    sequences, labels = [], []
    for i in range(len(dataframe) - seq_length):
        seq = dataframe.iloc[i:i+seq_length, :-1].values
        label = dataframe.iloc[i+seq_length-1, -1]
        sequences.append(seq)
        labels.append(label)
    return np.array(sequences), np.array(labels)

SEQ_LEN = 20
X, y = generate_sequences(df_processed, SEQ_LEN)

# === Step 4: Split into train/val/test (60/20/20) ===
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.3, random_state=42, stratify=y_temp) 


In [59]:


# === Step 5: Model definition ===
def build_advanced_lstm_model(input_shape):
    model = Sequential()
    model.add(Bidirectional(LSTM(128, return_sequences=True), input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))

    model.add(Bidirectional(LSTM(64, return_sequences=False)))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))

    model.add(Dense(128, kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4)))
    model.add(LeakyReLU(0.1))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Dense(64, kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4)))
    model.add(LeakyReLU(0.1))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Dense(32, kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4)))
    model.add(LeakyReLU(0.1))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))

    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = build_advanced_lstm_model((X_train.shape[1], X_train.shape[2]))


/home/purab/.local/lib/python3.12/site-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [60]:


# === Step 6: Callbacks ===
def create_callbacks():
    os.makedirs("models", exist_ok=True)
    return [
        EarlyStopping(monitor='val_accuracy', patience=30, restore_best_weights=True),
        ModelCheckpoint("models/best_classifier.keras", save_best_only=True, monitor='val_accuracy', mode='max'),
        ReduceLROnPlateau(monitor='val_accuracy', factor=0.2, patience=5, min_lr=1e-6, verbose=1),
        TensorBoard(log_dir='logs'),
        CSVLogger("training_log.csv")
    ]

# === Step 7: Train the model ===
history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=64,
    validation_data=(X_val, y_val),
    callbacks=create_callbacks(),
    verbose=1
)


Epoch 1/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 5s 52ms/step - accuracy: 0.5953 - loss: 0.7790 - val_accuracy: 0.5695 - val_loss: 0.7400 - learning_rate: 0.0010
Epoch 2/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - accuracy: 0.7189 - loss: 0.6357 - val_accuracy: 0.5695 - val_loss: 0.7341 - learning_rate: 0.0010
Epoch 3/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - accuracy: 0.7513 - loss: 0.5763 - val_accuracy: 0.5695 - val_loss: 0.7237 - learning_rate: 0.0010
Epoch 4/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - accuracy: 0.7752 - loss: 0.5280 - val_accuracy: 0.5762 - val_loss: 0.6855 - learning_rate: 0.0010
Epoch 5/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - accuracy: 0.7473 - loss: 0.5656 - val_accuracy: 0.7074 - val_loss: 0.6625 - learning_rate: 0.0010
Epoch 6/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - accuracy: 0.7828 - loss: 0.5208 - val_accuracy: 0.6289 - val_loss: 0.6515 - learning_rate: 0.0010
Epoch 7/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - accuracy: 0.7834 - loss: 0.5087 - 

In [61]:

y_pred = (model.predict(X_test) > 0.5).astype(int).flatten()

print("\nTest Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step

Test Accuracy: 0.8842530282637954
Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.92      0.90       423
           1       0.89      0.84      0.86       320

    accuracy                           0.88       743
   macro avg       0.88      0.88      0.88       743
weighted avg       0.88      0.88      0.88       743

